# LoRa SOM - 올바른 거리 공간에서의 분석

## 핵심 인사이트

LoRa 심볼의 본질:
- Dechirp 후 각 심볼은 **고유한 주파수의 순수 톤**
- 심볼 간 거리 = **주파수 인덱스의 순환적 거리**

```
Symbol k → Frequency bin k (after dechirp)
d(0, 1) = 1
d(0, 256) = 256 (최대)
d(0, 511) = 1 (순환: 512-511=1)
```

## 접근 방법
1. **원형 좌표**: 심볼 인덱스를 (cos, sin)으로 변환
2. **커스텀 거리**: 순환적 거리 함수 사용

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from scipy.signal import stft
import warnings
warnings.filterwarnings('ignore')

def _add_project_root_to_syspath() -> Path | None:
    here = Path.cwd().resolve()
    for p in (here, *here.parents):
        if (p / "utils" / "LoRa.py").exists():
            sys.path.insert(0, str(p))
            return p
    return None

PROJECT_ROOT = _add_project_root_to_syspath()
if PROJECT_ROOT is None:
    raise RuntimeError("Could not locate project root containing utils/LoRa.py")

from som import SelfOrganizingMap
from utils.LoRa import LoRa

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

print("모듈 임포트 완료!", "PROJECT_ROOT=", PROJECT_ROOT)

NameError: name 'Path' is not defined

## 1. LoRa 심볼 생성 및 분석

In [ ]:
SF = 9
BW = 125e3
OSF = 1

lora = LoRa(sf=SF, bw=BW, OSF=OSF)
num_symbols = 2 ** SF

# 심볼 생성
symbols = np.array([lora.gen_symbol(i) for i in range(num_symbols)])
labels = np.arange(num_symbols)

print(f"SF = {SF}, 심볼 개수 = {num_symbols}")

In [ ]:
# Dechirp 후 FFT 피크 위치 확인
peak_indices = []
for s in symbols:
    dechirped = lora.dechirp(s)
    fft_mag = np.abs(np.fft.fft(dechirped))
    peak_indices.append(np.argmax(fft_mag))

peak_indices = np.array(peak_indices)
print(f"피크 인덱스 == 심볼 인덱스: {np.all(peak_indices == labels)}")

## 2. 원형 좌표 변환

심볼 인덱스 k를 2D 원형 좌표로:
```
θ = 2π × k / N
x = cos(θ)
y = sin(θ)
```

In [ ]:
# 원형 좌표 생성
angles = 2 * np.pi * labels / num_symbols
circular_coords = np.column_stack([np.cos(angles), np.sin(angles)])

print(f"원형 좌표 shape: {circular_coords.shape}")

# 시각화
fig, ax = plt.subplots(figsize=(10, 10))
scatter = ax.scatter(circular_coords[:, 0], circular_coords[:, 1], 
                     c=labels, cmap='hsv', s=30, alpha=0.8)
plt.colorbar(scatter, label='Symbol Index')

# 일부 심볼 라벨
for idx in [0, 64, 128, 192, 256, 320, 384, 448]:
    ax.annotate(f'{idx}', circular_coords[idx], fontsize=10, fontweight='bold')

ax.set_xlabel('cos(θ)')
ax.set_ylabel('sin(θ)')
ax.set_title('LoRa Symbols in Circular Space')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 원형 좌표에서의 유클리드 거리 = 순환적 거리와 비례
print("원형 좌표에서의 유클리드 거리:")
for idx in [1, 10, 100, 256, 511]:
    d = np.linalg.norm(circular_coords[0] - circular_coords[idx])
    circular_d = min(idx, num_symbols - idx)  # 순환 거리
    print(f"  d(0, {idx:3d}) = {d:.4f}  (순환거리: {circular_d})")

## 3. SOM 학습 - 원형 좌표

In [ ]:
# 정규화 (원형 좌표는 이미 [-1, 1] 범위)
data = circular_coords.copy()

# SOM 파라미터
MAP_SIZE = (32, 32)  # 충분히 큰 맵
SIGMA = 16
LR = 0.5
ITERATIONS = 100000

print(f"SOM 설정: {MAP_SIZE}, σ={SIGMA}, lr={LR}, iter={ITERATIONS}")

In [ ]:
# SOM 생성 및 학습
som = SelfOrganizingMap(
    map_size=MAP_SIZE,
    input_dim=2,
    sigma=SIGMA,
    learning_rate=LR,
    distance_fn='euclidean',
    random_seed=42
)

# 데이터 기반 초기화
som.init_weights_from_data(data)

print("SOM 학습 시작...\n")
som.train(data, ITERATIONS, verbose=True)
print("\n완료!")

## 4. 결과 시각화

In [ ]:
# BMU 계산
bmu_positions = np.array([som.predict(x) for x in data])

In [ ]:
# U-Matrix
fig, ax = som.plot_u_matrix(figsize=(10, 8))
plt.title('U-Matrix (Circular Coordinates)')
plt.show()

In [ ]:
# 심볼 분포
fig, ax = plt.subplots(figsize=(14, 12))

u_matrix = som.get_u_matrix()
ax.imshow(u_matrix, cmap='gray_r', alpha=0.3, interpolation='nearest')

# jitter 없이 정확한 위치
scatter = ax.scatter(
    bmu_positions[:, 1],
    bmu_positions[:, 0],
    c=labels, cmap='hsv', alpha=0.8, s=60
)
plt.colorbar(scatter, label='Symbol Index')

ax.set_xlim(-0.5, MAP_SIZE[1]-0.5)
ax.set_ylim(MAP_SIZE[0]-0.5, -0.5)
ax.set_title('Circular Coordinates SOM - Symbol Distribution', fontsize=14)
ax.set_xlabel('SOM Column')
ax.set_ylabel('SOM Row')
plt.tight_layout()
plt.show()

In [ ]:
# 그룹별 시각화
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

group_size = num_symbols // 8

for i, ax in enumerate(axes.flat):
    start_idx = i * group_size
    end_idx = (i + 1) * group_size
    
    ax.imshow(u_matrix, cmap='gray_r', alpha=0.3, interpolation='nearest')
    
    mask = (labels >= start_idx) & (labels < end_idx)
    group_bmu = bmu_positions[mask]
    group_labels = labels[mask]
    
    sc = ax.scatter(
        group_bmu[:, 1],
        group_bmu[:, 0],
        c=group_labels, cmap='viridis', alpha=0.8, s=40
    )
    ax.set_xlim(-0.5, MAP_SIZE[1]-0.5)
    ax.set_ylim(MAP_SIZE[0]-0.5, -0.5)
    ax.set_title(f'Symbols {start_idx}-{end_idx-1}')

plt.suptitle('Symbol Groups on SOM', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 5. 토폴로지 분석

In [ ]:
# 연속 심볼 간 BMU 거리
consecutive_distances = []
for i in range(num_symbols):
    next_i = (i + 1) % num_symbols
    d = np.sqrt((bmu_positions[i,0] - bmu_positions[next_i,0])**2 + 
                (bmu_positions[i,1] - bmu_positions[next_i,1])**2)
    consecutive_distances.append(d)

consecutive_distances = np.array(consecutive_distances)

print("연속 심볼 간 BMU 거리:")
print(f"  평균: {consecutive_distances.mean():.3f}")
print(f"  표준편차: {consecutive_distances.std():.3f}")
print(f"  최소: {consecutive_distances.min():.3f}")
print(f"  최대: {consecutive_distances.max():.3f}")
print(f"  이웃(d≤√2) 비율: {(consecutive_distances <= np.sqrt(2)).mean()*100:.1f}%")

In [ ]:
# 거리 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(consecutive_distances, bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(np.sqrt(2), color='red', linestyle='--', label='√2 (인접 기준)')
axes[0].set_xlabel('BMU Distance')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Consecutive Symbol BMU Distance Distribution')
axes[0].legend()

axes[1].plot(consecutive_distances, 'b-', alpha=0.7)
axes[1].axhline(np.sqrt(2), color='red', linestyle='--')
axes[1].set_xlabel('Symbol Index')
axes[1].set_ylabel('Distance to Next Symbol')
axes[1].set_title('Consecutive Distance by Symbol Index')

plt.tight_layout()
plt.show()

In [ ]:
# 심볼 경로 (전체)
fig, ax = plt.subplots(figsize=(14, 12))

ax.imshow(u_matrix, cmap='gray_r', alpha=0.2, interpolation='nearest')

# 모든 연결선
for i in range(num_symbols):
    next_i = (i + 1) % num_symbols
    ax.plot([bmu_positions[i,1], bmu_positions[next_i,1]], 
            [bmu_positions[i,0], bmu_positions[next_i,0]], 
            'b-', alpha=0.2, lw=0.3)

scatter = ax.scatter(
    bmu_positions[:, 1],
    bmu_positions[:, 0],
    c=labels, cmap='hsv', s=30, alpha=0.9
)
plt.colorbar(scatter, label='Symbol Index')

ax.set_xlim(-0.5, MAP_SIZE[1]-0.5)
ax.set_ylim(MAP_SIZE[0]-0.5, -0.5)
ax.set_title('Complete Symbol Path on SOM', fontsize=14)
plt.tight_layout()
plt.show()

## 6. 최종 결과

In [ ]:
print("="*60)
print("최종 결과")
print("="*60)

print(f"\nQuantization Error: {som.quantization_error(data):.6f}")
print(f"Topographic Error: {som.topographic_error(data):.4f}")
print(f"연속 심볼 평균 BMU 거리: {consecutive_distances.mean():.3f}")
print(f"연속 심볼 이웃 비율: {(consecutive_distances <= np.sqrt(2)).mean()*100:.1f}%")

print("\n" + "="*60)
print("해석:")
print("- 원형 좌표는 심볼의 순환적 특성을 2D로 정확히 표현")
print("- 유클리드 거리가 심볼 간 실제 거리와 비례")
print("- 토폴로지 보존 = 색상 그라데이션 + 낮은 연속 심볼 거리")
print("="*60)